1.自定义层：
   * 带参数的层
   * 不带参数的层

***
2.文件读写
   * 加载和保存张量
   * 加载和保存模型参数

In [1]:
# 不带参数的层
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [2]:
# 验证一下
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [4]:
# 现在，我们可以将层作为组件合并到构建更复杂的模型中。
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [6]:
Y = net(torch.rand(4, 8))
Y.mean()
Y.shape

torch.Size([4, 128])

In [8]:
# 带参数的层

class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [9]:
# 实例化
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.6468, -0.3709,  0.8210],
        [ 0.0276, -0.4635, -0.7936],
        [-0.2383, -0.1799, -0.1329],
        [ 0.0879,  0.2967,  3.2110],
        [-2.8279,  1.3314, -0.0485]], requires_grad=True)

In [10]:
# 我们还可以使用自定义层构建模型
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

torch.Size([2, 1])

In [12]:
# 文件读写部分：
# 对于单个张量，我们可以直接调用load和save函数分别读写
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

tensor([0, 1, 2, 3])

In [13]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [14]:
# 我们可以存储一个张量列表，然后把它们读回内存。
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [15]:
# 写入或读取从字符串映射到张量的字典
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [17]:
# 加载和保存模型参数
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)
net.state_dict()

OrderedDict([('hidden.weight',
              tensor([[ 0.1414,  0.1394,  0.1123,  ...,  0.1445,  0.1443,  0.2183],
                      [ 0.0397,  0.2163, -0.1119,  ...,  0.0144,  0.0573, -0.1374],
                      [ 0.1329, -0.0665, -0.2027,  ...,  0.0693,  0.1316, -0.1639],
                      ...,
                      [ 0.0669, -0.1595,  0.1582,  ...,  0.0316, -0.1544,  0.1534],
                      [-0.1199,  0.1117,  0.1042,  ..., -0.0875, -0.0185, -0.0969],
                      [-0.1175, -0.0018,  0.0047,  ..., -0.2157,  0.1471,  0.0033]])),
             ('hidden.bias',
              tensor([ 0.0686,  0.1704,  0.0100,  0.1763,  0.1053, -0.0825,  0.0187,  0.0017,
                      -0.2133,  0.1802, -0.1089,  0.1716,  0.0216, -0.1982, -0.0934, -0.0977,
                       0.1516,  0.0691, -0.2066, -0.0339, -0.0680,  0.2016, -0.0111,  0.1381,
                       0.0830,  0.0088,  0.1984, -0.0877, -0.0884, -0.2093, -0.1989, -0.2088,
                      -0.0362,

In [18]:
# 我们将模型的参数存储为一个叫做“mlp.params”的文件。
torch.save(net.state_dict(), 'mlp.params')
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [19]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])